### CatBoost without preprocessing

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, precision_score, recall_score, mean_squared_error, r2_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import tree
from sklearn.model_selection import cross_val_score, cross_validate
import imblearn
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_text, plot_tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.feature_selection import VarianceThreshold
from lime.lime_tabular import LimeTabularExplainer
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv("data/Train.csv", index_col=0)
df.head()

,District,Block,CultLand,CropCultLand,LandPreparationMethod,CropTillageDate,CropTillageDepth,CropEstMethod,RcNursEstDate,SeedingSowingTransplanting,...,Harv_method,Harv_date,Harv_hand_rent,Threshing_date,Threshing_method,Residue_length,Residue_perc,Stubble_use,Acre,Yield
ID,,,,,,,,,,,,,,,,,,,,,
ID_GTFAC7PEVWQ9,Nalanda,Noorsarai,45,40,TractorPlough FourWheelTracRotavator,2022-07-20,5,Manual_PuddledRandom,2022-06-27,2022-07-21,...,machine,2022-11-16,NaN,2022-11-16,machine,30,40,plowed_in_soil,0.312500,600
ID_TK40ARLSPOKS,Nalanda,Rajgir,26,26,WetTillagePuddling TractorPlough FourWheelTrac...,2022-07-18,5,Manual_PuddledRandom,2022-06-20,2022-07-20,...,hand,2022-11-25,3.0,2022-12-24,machine,24,10,plowed_in_soil,0.312500,600
ID_1FJY2CRIMLZZ,Gaya,Gurua,10,10,TractorPlough FourWheelTracRotavator,2022-06-30,6,Manual_PuddledRandom,2022-06-20,2022-08-13,...,hand,2022-12-12,480.0,2023-01-11,machine,30,10,plowed_in_soil,0.148148,225
ID_I3IPXS4DB7NE,Gaya,Gurua,15,15,TractorPlough FourWheelTracRotavator,2022-06-16,6,Manual_PuddledRandom,2022-06-17,2022-07-17,...,hand,2022-12-02,240.0,2022-12-29,hand,26,10,plowed_in_soil,0.222222,468
ID_4T8YQWXWHB4A,Nalanda,Noorsarai,60,60,TractorPlough WetTillagePuddling,2022-07-19,4,Manual_PuddledRandom,2022-06-21,2022-07-20,...,machine,2022-11-30,NaN,2022-12-02,machine,24,40,plowed_in_soil,0.468750,550


In [3]:
# Calculate the yield per acre 
df['yield_per_acre'] = df['Yield'] / df['Acre'] 

In [4]:
# Drop the index column 
df = df.reset_index(drop=True)

In [5]:
# Step 1: Identify categorical columns
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_features = df.select_dtypes(exclude=['object', 'category']).columns.tolist()

In [6]:
# Step 2: Handle missing values
# For simplicity, let's use mean imputation for numeric columns and mode for categorical columns
numeric_imputer = SimpleImputer(strategy='mean')
df[numeric_features] = numeric_imputer.fit_transform(df[numeric_features])

categorical_imputer = SimpleImputer(strategy='most_frequent')
df[categorical_features] = categorical_imputer.fit_transform(df[categorical_features])

In [7]:
# Split the data 
X = df.drop(columns=['Yield', 'yield_per_acre'])  # Drop original Yield and yield_per_acre
y = df['yield_per_acre']  # Use yield_per_acre as the target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Step 5: Define the CatBoost Pools for training and testing, specifying categorical features
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=categorical_features)

In [9]:
# Step 6: Initialize and train the CatBoost regressor
model = CatBoostRegressor()
model.fit(train_pool, eval_set=test_pool, verbose=100)

Learning rate set to 0.060807
0:	learn: 1126.3159640	test: 1602.6493676	best: 1602.6493676 (0)	total: 74.7ms	remaining: 1m 14s
100:	learn: 890.2188518	test: 1491.9964920	best: 1491.9964920 (100)	total: 474ms	remaining: 4.22s
200:	learn: 772.9171656	test: 1465.6458742	best: 1465.4815885 (198)	total: 853ms	remaining: 3.39s
300:	learn: 694.4364068	test: 1460.9775381	best: 1460.7081975 (267)	total: 1.24s	remaining: 2.88s
400:	learn: 623.0555155	test: 1448.2798350	best: 1448.0000163 (399)	total: 1.61s	remaining: 2.41s
500:	learn: 562.7785679	test: 1441.1042412	best: 1441.1042412 (500)	total: 1.97s	remaining: 1.96s
600:	learn: 513.4566417	test: 1432.3550524	best: 1432.3550524 (600)	total: 2.32s	remaining: 1.54s
700:	learn: 475.4434430	test: 1430.8962638	best: 1429.5612631 (680)	total: 2.68s	remaining: 1.14s
800:	learn: 449.9997877	test: 1429.8834624	best: 1429.5612631 (680)	total: 3.04s	remaining: 755ms
900:	learn: 420.6080069	test: 1427.2664703	best: 1427.2664703 (900)	total: 3.4s	remaining

In [10]:
# Make predictions on the test set
y_pred = model.predict(test_pool)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate R-squared
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R2): {r2}")

Mean Absolute Error (MAE): 401.983480423031
Mean Squared Error (MSE): 2032915.781192621
Root Mean Squared Error (RMSE): 1425.8035563122364
R-squared (R2): 0.21070613106022062
